In [1]:
import numpy as np

In [2]:
def implicit_fin_diff(S,K,T,sigma,r,q,N,Nj,CallPut):    
    dt = T/N;    #dx = sigma*np.sqrt(3*dt)
    dx = 1.0/(2*Nj+1)
    nu = r - q - 0.5*sigma**2
    pu = -0.5*dt*((sigma/dx)**2 + nu/dx);   pm = 1.0 + dt*(sigma/dx)**2 + r*dt;    pd = -0.5*dt*((sigma/dx)**2 - nu/dx)
    grid = np.zeros(2*Nj+1)
    
    # Asset prices at maturity:
    St = [S*np.exp(-Nj*dx)]
    for j in range(1, 2*Nj+1):
        St.append(St[j-1]*np.exp(dx))
    
    # Option value at maturity:
    for j in range(2*Nj+1):
        if CallPut == 'Call':
            grid[j] = max(0, St[j] - K)
        elif CallPut == 'Put':
            grid[j] = max(0, K - St[j])
    
    # Boundary Conditions:
    if CallPut == 'Call':
        lambdaU = St[2*Nj] - St[2*Nj-1];    lambdaL = 0.0;
    elif CallPut == 'Put':
        lambdaU = 0.0;  lambdaL = -1.0*(St[1] - St[0])
    
    # Backwards computing through grid
    def tridiagonal(C,pU,pM,pD,lambda_L,lambda_U,nj):
        C1 = np.zeros(2*nj+1)     
        pmp = [pM+pD]
        pp = [C[1]+pD*lambda_L]
        for j in range(2,2*nj):
            pmp.append(pM - pU*pD/pmp[j-2])
            pp.append(C[j] - pp[j-2]*pD/pmp[j-2])
        C1[2*nj] = (pp[len(pp)-1] + pmp[len(pmp)-1]*lambda_U)/(pU + pmp[len(pmp)-1])
        C1[2*nj-1] = C1[2*nj] - lambda_U
        for j in range(2*nj-2, -1, -1):
            C1[j] = (pp[j-1] - pU*C1[j+1])/pmp[j-1]
        C1[0] = C1[1] - lambda_L
        return C1
    
    for i in range(N):  
        grid = tridiagonal(grid,pu,pm,pd,lambdaL,lambdaU,Nj)
    
    return grid[Nj]

In [3]:
Ans=implicit_fin_diff(40,40,1,0.3,0.1,0,400,400,'Put')
print(Ans)

2.8884560893904734


In [4]:
(self, S0, K, r, T, sigma, Smax, M, N, is_call=True)
(40, 40, 0.1, 1.0, 0.3, 100, 400, 400, False)

SyntaxError: invalid syntax (<ipython-input-4-1939bd7cec58>, line 1)